# This failed because its saved as python 2

In [1]:
import os
import pickle

try:
    izhi_opt = pickle.load(open('multi_objective_izhi.p','rb')) 
except:
    os.system('wget wget -O ncp_cell_layer_5_6.p https://osf.io/6yba2/download')
    os.system('wget -O multi_objective_izhi https://osf.io/3vp8d/download')
    izhi_opt = pickle.load(open('multi_objective_izhi.p','rb')) 


list(izhi_opt.values())[0]['pf'][0].dtc
list(izhi_opt.values())[0]['pf'][0].dtc.scores
list(izhi_opt.values())[0]['pf'][-1].dtc.scores
list(izhi_opt.values())[0]['pf'][2].dtc.scores
list(izhi_opt.values())[0]['pf'][1].dtc.scores
list(izhi_opt.values())[0]['pf'][1].dtc.vtest
list(izhi_opt.values())[0]['pf'][1].dtc.test
list(izhi_opt.values())[0]['pf'][1].dtc.tests
list(izhi_opt.values())[0]['pf'][1].dtc.tests[0].name


FileNotFoundError: [Errno 2] No such file or directory: 'multi_objective_izhi.p'

In [ ]:
#from manager import ModelManager
#from tables import Cells, Model_Waveforms, Models, Cell_Closest_Ephyz_Clusters
try:
    import manager
    reload(manager)
    from manager import ModelManager
except:
    !pip install hdbscan 


    

In [ ]:
!conda install numpy
!pip install pandas --upgrade

In [ ]:

from pandas import DataFrame
import numpy as np
import pandas
from matplotlib import pyplot as plt

import string 
from scipy.stats import pearsonr
import statsmodels.formula.api as smf
import hdbscan
from sklearn.cluster import KMeans
from scipy.spatial.distance import euclidean

pandas.set_option('display.max_columns', None)
pandas.set_option('display.max_rows', 20)
from manager import ModelManager
mgr = ModelManager()
mgr.server.connect()

In [ ]:
protocol_cost = {
    'Steady State':       1, # 1s
    'Standard':         5*2, # 5 repetitions of 1s SS 1s stim
    'Strong':           5*2, # 5 repetitions of 1s SS 1s strong stim
    'Input Resistance': 2*2, # 2 levels of 1s SS 1s stim
}

prop_protocol = {
    'AP1Amplitude': 'Standard',
    'AP2Amplitude': 'Standard',
    'AP12AmplitudeDrop': 'Standard',
    'AP12AmplitudeChangePercent': 'Standard',
    'AP1SSAmplitudeChange': 'Standard'  ,
    
    'AP1WidthHalfHeight': 'Standard',
    'AP2WidthHalfHeight': 'Standard',
    'AP12HalfWidthChangePercent': 'Standard',
    
    'AP1WidthPeakToTrough': 'Standard',
    'AP2WidthPeakToTrough': 'Standard',
    
    'AP1RateOfChangePeakToTrough': 'Standard',
    'AP2RateOfChangePeakToTrough': 'Standard', 
    'AP12RateOfChangePeakToTroughPercentChange': 'Standard',
    
    'AP1AHPDepth': 'Standard',
    'AP2AHPDepth': 'Standard',
    'AP12AHPDepthPercentChange': 'Standard',
    
    'AP1DelayMean': 'Standard',
    'AP2DelayMean': 'Standard',
    
    'AP1DelaySD': 'Standard',
    'AP2DelaySD': 'Standard',
    
    'AP1DelayMeanStrongStim': 'Strong',
    'AP2DelayMeanStrongStim': 'Strong',
    
    'AP1DelaySDStrongStim': 'Strong',
    'AP2DelaySDStrongStim': 'Strong',
    
    'Burst1ISIMean': 'Standard',
    'Burst1ISIMeanStrongStim': 'Strong',
    
    'Burst1ISISD': 'Standard',
    'Burst1ISISDStrongStim': 'Strong',
    
    'InitialAccommodationMean': 'Standard',
    'SSAccommodationMean': 'Standard',
    'AccommodationRateToSS': 'Standard',
    'AccommodationAtSSMean': 'Standard',
    'AccommodationRateMeanAtSS': 'Standard',
    
    
    'ISIMedian': 'Standard',
    'ISICV': 'Standard',
    'ISIBurstMeanChange': 'Standard',
    
    'SpikeRateStrongStim': 'Strong',
    
    'InputResistance': 'Input Resistance',
    
    'SteadyStateAPs': 'Steady State',
}

prop_names = [
    'AP1Amplitude',
    'AP2Amplitude',
    'AP12AmplitudeDrop',
    'AP12AmplitudeChangePercent',
    'AP1SSAmplitudeChange',  
    
    'AP1WidthHalfHeight',
    'AP2WidthHalfHeight',
    'AP12HalfWidthChangePercent',
    
    'AP1WidthPeakToTrough',
    'AP2WidthPeakToTrough',
    
    'AP1RateOfChangePeakToTrough',
    'AP2RateOfChangePeakToTrough',    
    'AP12RateOfChangePeakToTroughPercentChange',
    
    'AP1AHPDepth',
    'AP2AHPDepth',
    'AP12AHPDepthPercentChange',
    
    'AP1DelayMean',
    'AP2DelayMean',
    
    'AP1DelaySD',
    'AP2DelaySD',
    
    'AP1DelayMeanStrongStim',
    'AP2DelayMeanStrongStim',
    
    'AP1DelaySDStrongStim',
    'AP2DelaySDStrongStim',
    
    'Burst1ISIMean',
    'Burst1ISIMeanStrongStim',
    
    'Burst1ISISD',
    'Burst1ISISDStrongStim',
    
    'InitialAccommodationMean',
    'SSAccommodationMean',
    'AccommodationRateToSS',
    'AccommodationAtSSMean',
    'AccommodationRateMeanAtSS',
    
    
    'ISIMedian',
    'ISICV',
    'ISIBurstMeanChange',
    
    'SpikeRateStrongStim',
    
    'InputResistance',
    
    'SteadyStateAPs',
    
    'FrequencyPassAbove',
    'FrequencyPassBelow',
    
    'RampFirstSpike',
]

In [ ]:
# Get the remaining cell properties
cells = list(Cells\
    .select(Cells,Model_Waveforms.Spikes,Models.Name)\
    .join(Model_Waveforms, on=(Cells.Model_ID == Model_Waveforms.Model_id))\
    .join(Models, on=(Cells.Model_ID == Models.Model_ID))\
    .where((Model_Waveforms.Protocol == "STEADY_STATE") & (Model_Waveforms.Variable_Name == "Voltage"))\
    .order_by(Cells.Model_ID)
    .objects()
)

props = {}
for c, cell in enumerate(cells):
    for p, prop in enumerate(prop_names):
        if prop not in props:
            props[prop] = []
        
        if prop == 'SteadyStateAPs':
            props[prop].append(cell.Spikes)
            
        else:
            props[prop].append(getattr(cell, prop))
        
df = DataFrame(props, columns = prop_names)

model_ids = [c.Model_ID for c in cells]        
df.index = model_ids

In [ ]:
df.head()

In [ ]:
df['AP1Amplitude'].fillna(0, inplace=True)
df['AP2Amplitude'].fillna(0, inplace=True)

df['AP1SSAmplitudeChange'].fillna(0, inplace=True)

df['AP1WidthHalfHeight'].fillna(0, inplace=True)
df['AP2WidthHalfHeight'].fillna(0, inplace=True)

df['AP1WidthPeakToTrough'].fillna(0, inplace=True)
df['AP2WidthPeakToTrough'].fillna(0, inplace=True)

df['AP1RateOfChangePeakToTrough'].fillna(0, inplace=True)
df['AP2RateOfChangePeakToTrough'].fillna(0, inplace=True)

df['AP1AHPDepth'].fillna(0, inplace=True)
df['AP2AHPDepth'].fillna(0, inplace=True)

df['AP1DelayMean'].fillna(2000, inplace=True)
df['AP2DelayMean'].fillna(2000, inplace=True)

df['AP1DelaySD'].fillna(0, inplace=True)
df['AP2DelaySD'].fillna(0, inplace=True)

df['AP1DelayMeanStrongStim'].fillna(2000, inplace=True)
df['AP2DelayMeanStrongStim'].fillna(2000, inplace=True)

df['AP1DelaySDStrongStim'].fillna(0, inplace=True)
df['AP2DelaySDStrongStim'].fillna(0, inplace=True)

df['Burst1ISIMean'].fillna(2000, inplace=True)
df['Burst1ISIMeanStrongStim'].fillna(2000, inplace=True)

df['Burst1ISISD'].fillna(0, inplace=True)
df['Burst1ISISDStrongStim'].fillna(0, inplace=True)

df['AccommodationRateMeanAtSS'].fillna(2000, inplace=True)

df['ISIMedian'].fillna(2000, inplace=True)

df['ISICV'].fillna(0, inplace=True)

df['ISIBurstMeanChange'].fillna(0, inplace=True)

df['SpikeRateStrongStim'].fillna(0, inplace=True)

df['InputResistance'].fillna(df['InputResistance'].mean(), inplace=True)

df['FrequencyPassAbove'].fillna(29, inplace=True)
df['FrequencyPassBelow'].fillna(143, inplace=True)

df['RampFirstSpike'].fillna(5000, inplace=True)

for index, row in df.iterrows():
    
    # No APs
    if(row['AP1Amplitude'] == 0 and row['AP2Amplitude'] == 0):
        df.at[index, 'AP12AmplitudeDrop'] = 0
        df.at[index, 'AP12AmplitudeChangePercent'] = 0
        df.at[index, 'AP1SSAmplitudeChange'] = 0
        df.at[index, 'AP12HalfWidthChangePercent'] = 0
        df.at[index, 'AP12RateOfChangePeakToTroughPercentChange'] = 0
        df.at[index, 'AP12AHPDepthPercentChange'] = 0
        df.at[index, 'InitialAccommodationMean'] = 0
        df.at[index, 'SSAccommodationMean'] = 0
        df.at[index, 'AccommodationRateToSS'] = 0
        df.at[index, 'AccommodationAtSSMean'] = 0
        
    # Only 1 AP
    if(row['AP1Amplitude'] > 0 and row['AP2Amplitude'] == 0):
        df.at[index, 'AP12AmplitudeDrop'] = row['AP1Amplitude']
        df.at[index, 'AP12AmplitudeChangePercent'] = -100
        df.at[index, 'AP12HalfWidthChangePercent'] = -100
        df.at[index, 'AP12RateOfChangePeakToTroughPercentChange'] = -100
        df.at[index, 'AP12AHPDepthPercentChange'] = -100
        df.at[index, 'AccommodationRateToSS'] = -1
        df.at[index, 'AccommodationAtSSMean'] = -100
    
    # 1 AP and no SS APs
    if row['AP1SSAmplitudeChange'] == 0 and row['AP1Amplitude'] > 0:
        df.at[index, 'AP1SSAmplitudeChange'] = row['AP1Amplitude']
            
        
    if np.isnan(row['AccommodationRateToSS']):
        df.at[index, 'AccommodationRateToSS'] = -1
        
                
    if np.isnan(row['AccommodationAtSSMean']):
        df.at[index, 'AccommodationAtSSMean'] = -100

In [ ]:
def log_column(col):
    # Bi-symmetric log-like transformation, from: 
    # http://iopscience.iop.org/article/10.1088/0957-0233/24/2/027001/pdf
    trans = np.sign(df[col])*np.log(1+np.abs(df[col]*2.302585))
    df[col] = trans

In [ ]:
df_no_trans = df.copy()

log_column("AP12AmplitudeDrop")
log_column("AP12AmplitudeChangePercent")
log_column("AP1SSAmplitudeChange")
log_column("AP1WidthPeakToTrough")
log_column("AP2WidthPeakToTrough")
log_column("AP1RateOfChangePeakToTrough")
log_column("AP2RateOfChangePeakToTrough")
log_column("AP12RateOfChangePeakToTroughPercentChange")
log_column("AP12AHPDepthPercentChange")
log_column("AP1DelayMean")
log_column("AP2DelayMean")
log_column("AP1DelayMeanStrongStim")
log_column("AP2DelayMeanStrongStim")
log_column("Burst1ISIMean")
log_column("Burst1ISIMeanStrongStim")
log_column("AccommodationRateToSS")
log_column("AccommodationRateMeanAtSS")
log_column("ISIBurstMeanChange")
log_column("SpikeRateStrongStim")
log_column("InputResistance")
log_column("RampFirstSpike")
log_column("ISICV")
log_column("FrequencyPassAbove")
log_column("FrequencyPassBelow")

log_column("ISIMedian")
log_column("AP1WidthHalfHeight")
log_column("AP2WidthHalfHeight")
log_column("AP12HalfWidthChangePercent")
log_column("AP1AHPDepth")
log_column("AP2AHPDepth")

log_column("SteadyStateAPs")

In [ ]:
df_no_trans.to_csv("cell-ephyz-raw.csv")
df.to_csv("cell-ephyz-transformed.csv")

In [ ]:
df_all = df

In [ ]:
df.head()

In [ ]:
df_all["ClusterPath"] = "/"

In [ ]:
parent_path = "/3/1.0/"
#parent_path = "/1/1/"

# df = df_all

df = df_all[df_all["ClusterPath"].str.startswith(parent_path)]

# df = df_all[df_all["ClusterPath"].str.startswith("/0/")]
# df = df_all[df_all["Root_Cluster"] == 0]
# cluster_level = "Multi-Spikers" # 4 clusters

# df = df_all[df_all["ClusterPath"].str.startswith("/0/0/")]
# df = df_all[df_all["Multi-Spikers"] == 0] # 4 sub clusters - gain of 20  
# cluster_level = "Multi-Spikers-0"

#df = df_all[df_all["ClusterPath"].str.startswith("/0/0/0")]
#df = df_all[df_all["Multi-Spikers-0"] == 0] 
#cluster_level = "Multi-Spikers-0-0"

#df = df_all[df_all["ClusterPath"].str.startswith("/0/0/1")]
# df = df_all[df_all["Multi-Spikers-0"] == 1] 
# cluster_level = "Multi-Spikers-0-1"

#df = df_all[df_all["ClusterPath"].str.startswith("/0/0/2")] # NO MORE SUBCLUSTERS
#df = df_all[df_all["Multi-Spikers-0"] == 2] 
#cluster_level = "Multi-Spikers-0-2"

#df = df_all[df_all["ClusterPath"].str.startswith("/0/0/3")] # NO MORE SUBCLUSTERS
# df = df_all[df_all["Multi-Spikers-0"] == 3]  
# cluster_level = "Multi-Spikers-0-3"

# df = df_all[df_all["ClusterPath"].str.startswith("/0/2/")]
# df = df_all[df_all["Multi-Spikers"] == 2] # 2 sub clusters - gain of 17
# cluster_level = "Multi-Spikers-2"

# df = df_all[df_all["ClusterPath"].str.startswith("/0/1/")]
##df = df_all[df_all["Multi-Spikers"] == 1]  # no gains from subclustering
##cluster_level = "Multi-Spikers-1"

# df = df_all[df_all["ClusterPath"].str.startswith("/0/3/")]
##df = df_all[df_all["Multi-Spikers"] == 3]  # No gains from clustering further
##cluster_level = "Multi-Spikers-3"

#df = df_all[df_all["ClusterPath"].str.startswith("/0/1/3/")]

In [ ]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
x = ss.fit_transform(df.loc[:,prop_names].values)
x = DataFrame(x,columns=prop_names)
print('start dims', len(prop_names))
from sklearn.decomposition import PCA
pca = PCA(svd_solver='full',n_components=0.95)
principalComponents = pca.fit_transform(x)
principalDf = DataFrame(data = principalComponents)
X = principalDf
X.index = df.index
print('post-pca dims', len(principalDf.columns))
print(X.shape[0],'rows')


In [ ]:
%matplotlib inline

In [ ]:
pc = 0
#col = "AP12AmplitudeDrop"
#if col == "AP12AmplitudeDrop":
for col in df.columns:
    try:
        if np.abs(pearsonr(X[pc],df[col])[0]) >= 0:
            plt.scatter(X[pc],df[col])
            plt.title((col + str(pearsonr(X[pc],df[col]))))
            plt.show();
#             plt.scatter(X[pc],df_non_trans[col])
#             plt.show();
    except:
        print('failed',col)

In [ ]:
%matplotlib inline
from mpl_toolkits import mplot3d
plt.figure(figsize=(15, 7))  
plt.axes(projection='3d')
plt.plot(X[0],X[1], X[2],'bo')
plt.show()

from scipy.cluster.hierarchy import dendrogram, linkage  
from matplotlib import pyplot as plt

linked = linkage(X, 'ward',optimal_ordering=False)

plt.figure(figsize=(15, 7))  
dendrogram(linked,  
            orientation='top',
            distance_sort='acending',
            show_leaf_counts=True,
            #truncate_mode='lastp',
            #p=5,
            show_contracted=True,
          )
plt.ylim(0,160)
plt.show()  

print(X.shape[0],'items')

from sklearn.metrics import silhouette_samples, silhouette_score

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np

range_n_clusters = range(2, 8)

clusters = []
widths = []
for n_clusters in range_n_clusters:
    from sklearn.cluster import AgglomerativeClustering
    #clusterer = AgglomerativeClustering(n_clusters=n_clusters, affinity='euclidean', linkage='ward')  
    clusterer = KMeans(n_clusters=n_clusters)  
    cluster_labels = clusterer.fit_predict(X)

    silhouette_avg = silhouette_score(X, cluster_labels)
    print("For n_clusters =", n_clusters,
          "The average silhouette_score is :", silhouette_avg)

    # Compute the silhouette scores for each sample
    sample_silhouette_values = silhouette_samples(X, cluster_labels)

    y_lower = 10
    for i in range(n_clusters):
        ith_cluster_silhouette_values = \
            sample_silhouette_values[cluster_labels == i]

        ith_cluster_silhouette_values.sort()

        size_cluster_i = ith_cluster_silhouette_values.shape[0]
        y_upper = y_lower + size_cluster_i

        color = cm.nipy_spectral(float(i) / n_clusters)
    
    clusters.append(n_clusters)
    widths.append(silhouette_avg)

plt.plot(clusters, widths)
plt.show()

In [ ]:
%matplotlib inline
i=0
comp_names = []
for i in range(3):
    
    prop_r = np.array([stats.pearsonr(X[i],df[col])[0] if stats.pearsonr(X[i],df[col])[1] < 0.001 else 0 for col in df.columns[:-2]])
    inds = (-np.abs(prop_r)).argsort()
#     plt.plot(range(len(pca.components_[0])), pca.components_[i][inds])
#     plt.show()
    print(np.array(prop_names)[inds][:5])
    print(prop_r[inds][:5])
    
    name = ""
    for f in range(3):
        name += ("-" if prop_r[inds][f] < 0 else "+") + prop_names[inds[f]]
    comp_names.append(name)
    print(name)
    print("         -----            ")
    


In [ ]:
comp_names

In [ ]:
%matplotlib inline

In [ ]:
%matplotlib notebook
%matplotlib notebook

In [ ]:
min_cluster_size=10
n_clusters=6
hide_noise = False
remove_noise = False
k_means = True

X_w_noise = X

if remove_noise:
    #cluster = hdbscan.HDBSCAN(min_cluster_size=min_cluster_size)
    cluster.fit_predict(X)
    X = X[cluster.labels_ != -1]
    
if k_means:
    cluster = KMeans(n_clusters=n_clusters,random_state=1)
    cluster.fit_predict(X)

else:
    #cluster = hdbscan.HDBSCAN(min_cluster_size=min_cluster_size)
    cluster.fit_predict(X)

fig = plt.figure(figsize=(12, 10))
ax = fig.add_subplot(111, projection='3d')

if hide_noise:
    ax.scatter(
        X[cluster.labels_ != -1][0],
        X[cluster.labels_ != -1][1], 
        X[cluster.labels_ != -1][2], depthshade=False,marker='o', 
        c=cluster.labels_[cluster.labels_ != -1], 
        cmap='rainbow')
    
else:
    ax.scatter(
        X[0],
        X[1], 
        X[2], depthshade=False,marker='o', 
        c=cluster.labels_, 
        cmap='rainbow')

ax.set_xlabel(comp_names[0])
ax.set_ylabel(comp_names[1])
ax.set_zlabel(comp_names[2])

# ax.set_xlabel("AP Onset")
# ax.set_ylabel("SS Spike Rate")
# ax.set_zlabel("SS Accommodation")
plt.tight_layout()

centers = []

if k_means:
    centers = cluster.cluster_centers_
else:
    labels = np.unique(cluster.labels_) if not hide_noise else np.unique(cluster.labels_[cluster.labels_ != -1])
    
    for l in labels:
        X_label = X[cluster.labels_ == l]
        center = [np.mean(X_label[c]) for c in range(X.shape[1])]
        centers.append(center)

for i, center in enumerate(centers):
    ax.text(center[0],center[1],center[2],string.ascii_uppercase[i],size=20)
    
plt.show()
#%matplotlib inline

import collections
print(collections.Counter(cluster.labels_))


for c, center in enumerate(centers):
    dist = np.apply_along_axis(euclidean, 1, X, center)
    dist_sort_is = dist.argsort()
    from pprint import pprint as pp

    pp({"cluster": c, 
        "cells": X.iloc[dist_sort_is].index[:5], 
        "sd":["{:12.2f}".format(np.std(X.iloc[np.where(cluster.labels_ == c)][pc])) for pc in range(3)],
        "center":["{:12.2f}".format(c) for c in center[0:3]],
       })

In [ ]:
comp_names

In [ ]:
source_df.head()

In [ ]:
%matplotlib %notebook

In [ ]:
source_df = df_no_trans.ix[X.index]

display_props = ["ISIMedian","AccommodationAtSSMean","AP1DelayMeanStrongStim"]
#display_props = ["AP1DelayMeanStrongStim","ISIMedian","AccommodationAtSSMean"]

fig = plt.figure(figsize=(12, 10))
ax = fig.add_subplot(111, projection='3d')

ax.scatter(
    source_df[display_props[0]],
    source_df[display_props[1]], 
    source_df[display_props[2]], 
    depthshade=True,
    marker='o', 
    c=cluster.labels_, 
    cmap='rainbow')

ax.set_xlabel(display_props[0])
ax.set_ylabel(display_props[1])
ax.set_zlabel(display_props[2])

plt.tight_layout()

centers = []
sds = []

labels = np.unique(cluster.labels_)

print(display_props)

for i, l in enumerate(labels):
    X_label = source_df[cluster.labels_ == l]
    center = [np.mean(X_label[prop]) for prop in display_props]
    centers.append(center)
    
    sd = [np.std(X_label[prop]) for prop in display_props]
    sds.append(sd)
    
    ax.text(center[0],center[1],center[2],string.ascii_uppercase[i],size=20)
    
    print(string.ascii_uppercase[i],["{:0.2f}+/-{:0.2f}".format(centers[i][c],sds[i][c]) for c,_ in enumerate(center)])
    
    reg = smf.ols('AP1DelayMeanStrongStim~AccommodationAtSSMean+ISIMedian',data=X_label).fit()
    print('reg params',reg._results.params)
    print('reg p vals',reg._results.pvalues)
    print("delay v accom",stats.pearsonr(X_label["AP1DelayMeanStrongStim"],X_label["AccommodationAtSSMean"]))
    print("delay v isi",stats.pearsonr(X_label["AP1DelayMeanStrongStim"],X_label["ISIMedian"]))
    
    
plt.show()        

#%matplotlib inline


In [ ]:
dir(smf.ols('AP1DelayMeanStrongStim~AccommodationAtSSMean+ISIMedian',data=X_label).fit()._results)

In [ ]:
comp_names

In [ ]:
X_w_labels = DataFrame(X)

In [ ]:
X["Cluster"] = cluster.labels_

In [ ]:
#X_w_noise.loc[X.index]['Cluster']#=cluster.labels_

for label in X.index:
    X_w_noise.at[label, "Cluster"] = X.at[label, "Cluster"]
    

In [ ]:
np.unique(X_w_noise["Cluster"])

           delay       short aps/ISIs       accom    


2 ['        1.37', '        1.29', '        2.55'] large accomodotation, mild delay, short aps
regular spikers

4 ['        2.97', '        1.24', '       -1.17'] delayed, regular spiking, mild accomodating
non-accomodating regular spikers



0 ['        3.43', '       -2.73', '       -0.49'] delayed, tall APs, long isis
slow spikers

3 ['       -1.98', '       -2.06', '        0.96'] non delayed, long isis, accommodating
rapid onset slow spikers


5 ['       -3.69', '        0.71', '        1.40'] rapid onset+accomodation (=burst?), reg? spiking
bursters


1 ['       -1.90', '        2.06', '       -2.26'] non delayed, non-accomodating, short aps, short isis
rapid onset, non-accomodoating, fast spikers



within clusters:

correlation between 
	accom and short ap/isis
	short ap/isis and ap delay


In [ ]:
df["Cluster"] = X_w_noise["Cluster"]

df["ClusterPath"] = parent_path + df["Cluster"].map(str) + "/"

for label in df.index:
    df_all.at[label, "ClusterPath"] = df.at[label, "ClusterPath"]
    df_all.at[label, "Cluster"] = df.at[label, "Cluster"]

np.unique(df["ClusterPath"])

In [ ]:
np.unique(df_all["ClusterPath"])

# End clustering

In [ ]:
cl = 0
cluster_rows = X[cluster.labels_ == cl]
plt.scatter(cluster_rows[1],cluster_rows[2])
plt.show()
from scipy import stats
def corrfunc(x, y, **kws):
    r, p = stats.pearsonr(x, y)
    ax = plt.gca()
    ax.annotate("r = {:.2f}, p = {:.3f}".format(r,p),
                xy=(.1, .9), xycoords=ax.transAxes)
    
import seaborn as sns
g = sns.PairGrid(df_all.ix[cluster_rows.index],vars=["ISIMedian","SSAccommodationMean","AccommodationAtSSMean","InitialAccommodationMean"])
g.map_upper(plt.scatter, s=10)
g.map_lower(plt.scatter, s=10)
g.map_lower(corrfunc)

In [ ]:
cluster_rows[cluster_rows[1].min() == cluster_rows[1]]

In [ ]:
cell_records = list(Cells\
    .select(Cells,Model_Waveforms.Spikes,Models.Name)\
    .join(Model_Waveforms, on=(Cells.Model_ID == Model_Waveforms.Model_id))\
    .join(Models, on=(Cells.Model_ID == Models.Model_ID))\
    .where((Model_Waveforms.Protocol == "STEADY_STATE") & (Model_Waveforms.Variable_Name == "Voltage"))\
    .order_by(Cells.Model_ID)
)

In [ ]:
df_all.loc["NMLCL001124"]["ClusterPath"]

In [ ]:
for cell_id in df_all.index:
    cell = next(cell for cell in cell_records if cell.Model_ID == cell_id)
    cell.ClusterPath = df_all.loc[cell_id]["ClusterPath"]

In [ ]:
np.unique([cell.ClusterPath for cell in cell_records])

In [ ]:
X_w_noise

In [ ]:
isimedian ~ delays + accomodations

ISIMedian  
AP1Amplitude
AP2Amplitude

AP1DelayMean
AP2DelayMean

InitialAccommodationMean
SSAccommodationMean
AccommodationAtSSMean

In [ ]:
sns.pairplot(cl4_raw,vars=[
    "ISIMedian",
"AP1Amplitude",
"AP1DelayMean",
"InitialAccommodationMean",
"SSAccommodationMean",
"AccommodationAtSSMean"])

In [ ]:
smf.ols('AP1DelayMean~InitialAccommodationMean+SSAccommodationMean+AccommodationAtSSMean+ISIMedian',data=cl4_raw).fit().summary()

In [ ]:
plt.hist(cl4_raw[col])
plt.show()

In [ ]:
%matplotlib inline
cols = ["ISIMedian"]

for cl in range(6):
    print("Cluster", cl)
    cl4 = np.where(cluster.labels_ == cl)
    cl4_data = X.iloc[cl4]
    
    cl4_data.columns = ["PC" + str(c) for c in cl4_data.columns]
    cl4_raw = df_all.loc[cl4_data.index]
    
    
    for col in cols:
        plt.scatter(cl4_raw["AP1DelayMean"], cl4_raw[col]); plt.show();
        print("AP1DelayMean",col,pearsonr(cl4_raw["AP1DelayMean"], cl4_raw[col]))
        print(smf.ols('AP1DelayMean~'+col,data=cl4_raw).fit().summary())
    print("---")

In [ ]:
fr._results.params[1]

In [ ]:
import collections
collections.Counter(df_all["ClusterPath"])

In [ ]:
X_w_noise_clusters = []
X_w_noise_cluster_dists = []
for row_id in X_w_noise.index:
    cl_dists = []
    for cl, center in enumerate(cluster.cluster_centers_):
        cl_dists.append(euclidean(X_w_noise.loc[row_id], center))
    dist_sorted = np.argsort(cl_dists)
    X_w_noise_clusters.append(dist_sorted[0:3])
    X_w_noise_cluster_dists.append(np.array(cl_dists)[dist_sorted[0:3]])
X_w_noise_clusters = np.array(X_w_noise_clusters)
X_w_noise_clusters

In [ ]:
X_w_noise_clusters

In [ ]:
record.Cell_ID

In [ ]:
for i, cell_id in enumerate(X_w_noise.index):
    print(cell_id)
    for cli, cl in enumerate(X_w_noise_clusters[i]):
        record = Cell_Closest_Ephyz_Clusters()
        record.Cell_ID = cell_id
        record.Cluster_ID = parent_path + str(cl) + "/"
        record.Distance = X_w_noise_cluster_dists[i][cli]
        record.save()

In [ ]:
from database import NMLDB

In [ ]:
db = NMLDB()
db.connect()

In [ ]:
%matplotlib notebook
%matplotlib notebook

In [ ]:
fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111, projection='3d')

ax.scatter(
        X_w_noise[0],
        X_w_noise[1], 
        X_w_noise[2], depthshade=False,marker='o', 
        c=X_w_noise_clusters[:,0], 
        cmap='rainbow')

ax.set_xlabel(comp_names[0])
ax.set_ylabel(comp_names[1])
ax.set_zlabel(comp_names[2])

plt.tight_layout()

for i, center in enumerate(cluster.cluster_centers_):
    ax.text(center[0],center[1],center[2],str(i),size=20)

plt.show()

In [ ]:
#%matplotlib inline
from scipy.stats import pearsonr
cols = [
"PC0",
#"PC0"
]

for cl in range(6):
    print("Cluster", cl)
    cl_ids = np.where(X_w_noise_clusters[:,0] == cl)
    cl_data = X_w_noise.iloc[cl_ids]
    
    cl_data.columns = ["PC" + str(c) for c in cl_data.columns]
    cl_raw = df_all.loc[cl_data.index]
    
    
    for col in cols:
        corr = pearsonr(cl_data["PC1"], cl_data[col])
        
        #plt.scatter(cl_data["PC1"], cl_data[col]);plt.show();
        fr = smf.ols('PC1~'+col,data=cl_data).fit()
        print("PC1",col,corr[0],corr[1] < 0.05)
        print('coeff',fr._results.params[1])
        #print(fr.summary())
    print("---")

In [ ]:

from scipy.stats import pearsonr
y = "AP1DelayMeanStrongStim"
cols = [
"ISIMedian",
#"PC0"
]

for cl in range(6):
    print("Cluster", cl)
    cl_ids = np.where(X_w_noise_clusters[:,0] == cl)
    cl_data = X_w_noise.iloc[cl_ids]
    
    cl_data.columns = ["PC" + str(c) for c in cl_data.columns]
    cl_raw = df_all.loc[cl_data.index]
    
    
    for col in cols:
        corr = pearsonr(cl_raw[y], cl_raw[col])
        
        #%matplotlib inline
        #plt.scatter(cl_raw[y], cl_raw[col]);plt.show();
        fr = smf.ols(y+'~'+col,data=cl_raw).fit()
        print(y,col,corr[0],corr[1] < 0.05)
        print('coeff',fr._results.params[1])
        #print(fr.summary())
    print("---")